[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jeremylongshore/claude-code-plugins-plus-skills/blob/main/tutorials/skills/02-skill-anatomy.ipynb)

# Skill Anatomy: Deep Dive into SKILL.md Structure

**Learning Path**: Skills → Plugins → Orchestration  
**Level**: Beginner-Intermediate  
**Time**: 30 minutes  
**Prerequisites**: [01-what-is-skill.ipynb](01-what-is-skill.ipynb)

---

## What You'll Learn

- ✅ Complete SKILL.md frontmatter structure
- ✅ Required vs optional fields (Enterprise mode)
- ✅ Body structure and best practices
- ✅ Tool authorization patterns
- ✅ Model selection strategies
- ✅ Build and validate real skills

---

## SKILL.md: Two Parts

Every SKILL.md has exactly **two sections**:

```markdown
---
[FRONTMATTER: YAML configuration]
---

[BODY: Markdown instructions]
```

### Part 1: Frontmatter (YAML)

**What it tells Claude**:
- **Name**: What to call this skill
- **Description**: When to use it (triggers + use cases)
- **Tools**: Which tools are pre-authorized
- **Model**: Which Claude model to use (optional override)
- **Metadata**: Version, author, license, tags

### Part 2: Body (Markdown)

**What it tells Claude**:
- **How** to execute the skill
- Step-by-step instructions
- Examples and templates
- Error handling
- Output format

In [ ]:
# Visualize the structure
structure = {
    "FRONTMATTER (YAML)": {
        "tells_claude": "WHAT and WHEN",
        "fields": [
            "name (required)",
            "description (required)",
            "allowed-tools (required, CSV)",
            "version (required)",
            "license (required)",
            "author (required)",
            "tags (required)",
            "model (optional)",
            "disable-model-invocation (optional)"
        ]
    },
    "BODY (Markdown)": {
        "tells_claude": "HOW to execute",
        "contains": [
            "Step-by-step procedure",
            "Examples",
            "Templates",
            "Error handling",
            "Output requirements"
        ]
    }
}

print("SKILL.MD ANATOMY")
print("=" * 60)
for section, details in structure.items():
    print(f"\n{section}")
    print(f"  Purpose: {details['tells_claude']}")
    
    key = 'fields' if 'fields' in details else 'contains'
    print(f"  {key.title()}:")
    for item in details[key]:
        print(f"    • {item}")

print("\n" + "=" * 60)
print("💡 Frontmatter = Configuration, Body = Instructions")

---

## Frontmatter: Required Fields (Enterprise Mode)

**Enterprise Mode**: ALL fields below are REQUIRED (no exceptions)

### 1. `name` (String)

**Format**: `kebab-case` (lowercase, hyphens only)
**Pattern**: `^[a-z0-9-]+$`
**Max length**: 64 characters
**Reserved**: Cannot contain "claude" or "anthropic"

```yaml
name: bigquery-migrations      ✅ CORRECT
name: BigQueryMigrations       ❌ WRONG (camelCase)
name: bigquery_migrations      ❌ WRONG (snake_case)
name: claude-helper            ❌ WRONG (reserved word)
```

### 2. `description` (String, Multiline)

**Must include**:
1. What the skill does (1 sentence)
2. "Use when..." phrase (triggers automatic invocation)
3. 2-6 trigger phrases (helps Claude match intent)

```yaml
description: |
  Generate BigQuery schema migrations with validation.
  Use when: migrating schemas, adding fields, changing types.
  Triggers: migration, schema change, add field, alter table.
```

### 3. `allowed-tools` (String, CSV)

**CRITICAL**: Must be CSV string, NOT YAML array

```yaml
allowed-tools: "Read,Write,Grep,Bash(git:*)"  ✅ CORRECT (CSV)
allowed-tools: [Read, Write, Grep]             ❌ WRONG (array)
```

**Bash scoping is MANDATORY**:
```yaml
allowed-tools: "Bash(git:*),Bash(npm:*)"  ✅ CORRECT (scoped)
allowed-tools: "Bash"                     ❌ WRONG (unscoped)
```

### 4. `version` (String, SemVer)

**Format**: `MAJOR.MINOR.PATCH` (3 parts)

```yaml
version: 1.0.0      ✅ CORRECT
version: 1.0        ❌ WRONG (only 2 parts)
version: "1.0.0"    ✅ CORRECT (quoted is fine)
```

### 5. `license` (String, SPDX)

```yaml
license: MIT
license: Apache-2.0
license: GPL-3.0
```

### 6. `author` (String)

**Format**: `Name <email>`

```yaml
author: Jane Developer <jane@example.com>  ✅ CORRECT
author: Jane Developer                      ❌ WRONG (no email)
```

### 7. `tags` (Array)

**Purpose**: Categorization and discovery

```yaml
tags: ["database", "bigquery", "migration", "schema"]
```

In [ ]:
# Build a valid frontmatter
def build_frontmatter(name, description, tools, version="1.0.0", 
                     license="MIT", author="Developer <dev@example.com>",
                     tags=None):
    """Build enterprise-compliant SKILL.md frontmatter"""
    
    if tags is None:
        tags = ["general"]
    
    frontmatter = f"""---
name: {name}
description: |
  {description}
allowed-tools: "{tools}"
version: {version}
license: {license}
author: {author}
tags: {tags}
---"""
    
    return frontmatter

# Example: Build a skill frontmatter
example = build_frontmatter(
    name="code-reviewer",
    description="""Review code for quality, security, and best practices.
  Use when: reviewing pull requests, auditing code, checking standards.
  Triggers: review code, code quality, security audit, check pr.""",
    tools="Read,Grep,Bash(git:*)",
    tags=["code-review", "quality", "security"]
)

print("GENERATED FRONTMATTER:")
print("=" * 60)
print(example)
print("=" * 60)
print("\n✅ This follows all enterprise standards!")

---

## Frontmatter: Optional Fields

### `model` (String)

**Override default model for this skill**

```yaml
model: sonnet    # Use Claude Sonnet
model: haiku     # Use Claude Haiku (faster, cheaper)
```

**When to use**:
- `haiku`: Simple, deterministic tasks (fast)
- `sonnet`: Complex reasoning, analysis (default)
- ❌ `opus`: DEPRECATED - do not use

### `disable-model-invocation` (Boolean)

**Require manual invocation** (no auto-discovery)

```yaml
disable-model-invocation: true
```

**When to use**:
- Dangerous operations (destructive commands)
- Requires explicit user confirmation
- User must type `/skill-name` to invoke

In [ ]:
# Tool authorization patterns
tool_patterns = {
    "Read-only analysis": "Read,Grep,Glob",
    
    "File manipulation": "Read,Write,Edit,Grep,Glob",
    
    "Git operations": "Read,Write,Bash(git:*)",
    
    "Package management": "Read,Bash(npm:*),Bash(pip:*)",
    
    "Database operations": "Read,Write,Bash(psql:*),Bash(mysql:*)",
    
    "Full access (dangerous)": "Read,Write,Edit,Grep,Glob,Bash(*:*)",
}

print("COMMON TOOL AUTHORIZATION PATTERNS")
print("=" * 60)
for use_case, tools in tool_patterns.items():
    print(f"\n{use_case}:")
    print(f"  allowed-tools: \"{tools}\"")

print("\n" + "=" * 60)
print("💡 Principle: Grant minimum tools needed")
print("   Read-only skills don't need Write/Edit")
print("   ALWAYS scope Bash: Bash(git:*), never just Bash")

---

## Body Structure: Best Practices

### Size Limits (Enterprise)

**SKILL.md body MUST be**:
- ≤ 5,000 words
- ≤ 500 lines
- ≤ ~7,500 tokens

**Why**: Context budget optimization

**If too large**: Move content to `references/` directory

### Recommended Structure

```markdown
# [Skill Name]

## What This Skill Does

Brief overview (2-3 sentences)

## When to Use This Skill

- Use case 1
- Use case 2
- Use case 3

## Prerequisites

What must exist before running (files, env vars, etc.)

## Steps

### Step 1: [Action]

Detailed instructions...

### Step 2: [Action]

Detailed instructions...

## Output Format

What the skill should produce (JSON, files, etc.)

## Error Handling

How to handle common errors

## Examples

Working examples of skill execution
```

In [ ]:
# Build a complete SKILL.md
def build_complete_skill(name, description, tools, body_content,
                         version="1.0.0", license="MIT",
                         author="Developer <dev@example.com>",
                         tags=None, model=None):
    """Build a complete enterprise-compliant SKILL.md file"""
    
    if tags is None:
        tags = ["general"]
    
    # Build frontmatter
    frontmatter = f"""---
name: {name}
description: |
  {description}
allowed-tools: "{tools}"
version: {version}
license: {license}
author: {author}
tags: {tags}"""
    
    if model:
        frontmatter += f"\nmodel: {model}"
    
    frontmatter += "\n---"
    
    # Combine frontmatter + body
    complete_skill = f"{frontmatter}\n\n{body_content}"
    
    return complete_skill

# Example: Build a complete skill
body = """# Git Commit Message Generator

## What This Skill Does

Generates conventional commit messages following team standards.

## Steps

### Step 1: Analyze Changes
Use `git diff --staged` to see what changed.

### Step 2: Determine Type
- feat: New feature
- fix: Bug fix
- docs: Documentation
- refactor: Code refactoring

### Step 3: Write Message
Format: `type(scope): description`

## Output Format
Return the commit message as plain text.
"""

skill = build_complete_skill(
    name="git-commit-helper",
    description="""Generate conventional commit messages.
  Use when: creating commits, writing commit messages.
  Triggers: commit message, generate commit, conventional commits.""",
    tools="Read,Bash(git:*)",
    body_content=body,
    tags=["git", "commits", "productivity"],
    model="haiku"  # Fast for simple task
)

print("COMPLETE SKILL.MD:")
print("=" * 60)
print(skill)
print("=" * 60)
print("\n✅ Ready to save as SKILL.md!")

---

## 🎯 Try It: Validate Your Skill

In [ ]:
import re

def validate_skill_enterprise(skill_content):
    """Validate skill against enterprise standards (6767-c)"""
    errors = []
    warnings = []
    
    # Split frontmatter and body
    parts = skill_content.split('---')
    if len(parts) < 3:
        errors.append("CRITICAL: Invalid format - need frontmatter between ---")
        return errors, warnings
    
    frontmatter_raw = parts[1].strip()
    body = '---'.join(parts[2:]).strip()
    
    # Parse frontmatter (simple YAML)
    frontmatter = {}
    for line in frontmatter_raw.split('\n'):
        if ':' in line and not line.startswith(' '):
            key, value = line.split(':', 1)
            frontmatter[key.strip()] = value.strip()
    
    # Required fields
    required = ["name", "description", "allowed-tools", "version", "license", "author", "tags"]
    for field in required:
        if field not in frontmatter:
            errors.append(f"CRITICAL: Missing required field '{field}'")
    
    # Name validation
    if "name" in frontmatter:
        name = frontmatter["name"].strip('"')
        if not re.match(r'^[a-z0-9-]+$', name):
            errors.append(f"CRITICAL: Name must be kebab-case: {name}")
        if "claude" in name or "anthropic" in name:
            errors.append(f"CRITICAL: Name cannot contain 'claude' or 'anthropic'")
    
    # allowed-tools validation
    if "allowed-tools" in frontmatter:
        tools = frontmatter["allowed-tools"]
        if not tools.startswith('"'):
            errors.append("CRITICAL: allowed-tools must be quoted CSV string")
        if "Bash" in tools and "Bash(" not in tools:
            errors.append("CRITICAL: Bash must be scoped (e.g., Bash(git:*))")
    
    # Version validation
    if "version" in frontmatter:
        version = frontmatter["version"].strip('"')
        if not re.match(r'^\d+\.\d+\.\d+$', version):
            errors.append(f"CRITICAL: Version must be SemVer (x.y.z): {version}")
    
    # Model validation
    if "model" in frontmatter:
        model = frontmatter["model"].strip('"')
        if model == "opus":
            errors.append("CRITICAL: Model 'opus' is deprecated - use 'sonnet' or 'haiku'")
    
    # Body size check
    lines = body.split('\n')
    words = body.split()
    if len(lines) > 500:
        warnings.append(f"HIGH: Body has {len(lines)} lines (max 500)")
    if len(words) > 5000:
        warnings.append(f"HIGH: Body has {len(words)} words (max 5000)")
    
    return errors, warnings

# Test with our example
errors, warnings = validate_skill_enterprise(skill)

print("VALIDATION RESULTS")
print("=" * 60)
if not errors and not warnings:
    print("✅ PASSED - Skill follows all enterprise standards!")
else:
    if errors:
        print("\n❌ CRITICAL ERRORS:")
        for error in errors:
            print(f"   {error}")
    if warnings:
        print("\n⚠️  WARNINGS:")
        for warning in warnings:
            print(f"   {warning}")

print("\n" + "=" * 60)

---

## Key Takeaways

### Frontmatter (YAML)

1. ✅ **7 required fields**: name, description, allowed-tools, version, license, author, tags
2. ✅ **Name**: kebab-case, no reserved words
3. ✅ **allowed-tools**: CSV string (not array), Bash must be scoped
4. ✅ **Version**: SemVer (x.y.z)
5. ✅ **Description**: Include "Use when..." and trigger phrases

### Body (Markdown)

1. ✅ **Size limits**: ≤500 lines, ≤5000 words, ≤7500 tokens
2. ✅ **Structure**: What/When/Prerequisites/Steps/Output/Errors/Examples
3. ✅ **Clarity**: Step-by-step, no ambiguity
4. ✅ **References**: Heavy content in `references/` directory

### Best Practices

- 🔒 **Minimum tools**: Grant only what's needed
- 🚀 **Model selection**: `haiku` for simple, `sonnet` for complex
- 📏 **Keep it focused**: One skill = one clear purpose
- 🧪 **Test thoroughly**: Validate before deployment

---

## Next Steps

**You now understand skill anatomy!**

1. **[03-build-your-first-skill.ipynb](03-build-your-first-skill.ipynb)** - Create a real skill from scratch
2. **[04-advanced-patterns.ipynb](04-advanced-patterns.ipynb)** - Multi-phase workflows
3. **[05-skill-standards.ipynb](05-skill-standards.ipynb)** - Full validation system

---

*Enterprise Standards Compliant • Version 1.0.0 • MIT License*